In [1]:
import jax.numpy as jnp
import jax
import numpy as np
import time
import gymnasium as gym
import exciting_environments as excenv
#from exciting_environments import GymWrapper, EnvironmentRegistry
#import jax_dataclasses as jdc
from dataclasses import fields
from exciting_environments.utils import MinMaxNormalization
jax.config.update("jax_enable_x64", True)

/home/olivers/exc_env/exciting-environments/exciting_environments/pmsm/motor_parameters.py:115: UserWarning: A JAX array is being set as static! This can result in unexpected behavior and is usually a mistake to do.
  BRUSA = MotorParams(
/home/olivers/exc_env/exciting-environments/exciting_environments/pmsm/motor_parameters.py:147: UserWarning: A JAX array is being set as static! This can result in unexpected behavior and is usually a mistake to do.
  SEW = MotorParams(
/home/olivers/exc_env/exciting-environments/exciting_environments/pmsm/motor_parameters.py:174: UserWarning: A JAX array is being set as static! This can result in unexpected behavior and is usually a mistake to do.
  DEFAULT = MotorParams(


Failed to import warp: No module named 'warp'
Failed to import mujoco_warp: No module named 'warp'


In [2]:
# pmsm_env=EnvironmentRegistry.PMSM.make(saturated=True,motor_variant=excenv.pmsm.MotorVariant.BRUSA,batch_size=2)
# pmsm_env2=EnvironmentRegistry("PMSM-v0").make(saturated=True,motor_variant=excenv.pmsm.MotorVariant("BRUSA"),batch_size=2)
#env=excenv.make('MassSpringDamper-v0',batch_size=2)

In [2]:
pend_env=excenv.PMSM(saturated=False,LUT_motor_name="BRUSA")

In [3]:
key=jax.random.PRNGKey(1234)
obs, state = pend_env.reset(key)
generated_observations = []
generated_actions= []
generated_observations.append(obs)
for i in range(100):
    key,subkey= jax.random.split(key)
    action = jax.random.uniform(subkey,(2,),minval=-1,maxval=1)
    obs, state = pend_env.step(state, action)
    generated_actions.append(action)
    generated_observations.append(obs)
generated_observations = jnp.array(generated_observations)
generated_actions = jnp.array(generated_actions)

In [4]:
key=jax.random.PRNGKey(123)
obs, state = pend_env.reset(key)

In [25]:

keys = jax.random.split(jax.random.PRNGKey(0), 2)
#envs = [Pendulum(solver=diffrax.Euler(),batch_size=1,static_params={"g": jnp.array(9.81), "l": jnp.array(float(1+i)),  "m": jnp.array(1.0)}) for i in range(4)]
envs = [excenv.PMSM(saturated=True,LUT_motor_name="BRUSA"),excenv.PMSM(saturated=True,LUT_motor_name="BRUSA")]
batched_envs = jax.tree.map(lambda *args: jnp.stack(args), *envs)
#batched_envs.reset(rng=jax.random.PRNGKey(321))

In [26]:
keys = jax.random.split(jax.random.PRNGKey(0), 2)
obs, states = jax.vmap(lambda e,k: e.reset())(batched_envs,keys)

In [27]:
actions = jnp.ones((2,2))

next_obs, next_states = jax.vmap(
    lambda e, s, a: e.step(s, a)
)(batched_envs, states, actions)
next_obs

Array([[-0.02880888, -0.0114613 ,  0.        , -0.03680053,  0.98510933,
         0.1719291 ,  0.70525317,  0.70895558],
       [-0.02880888, -0.0114613 ,  0.        , -0.03680053,  0.98510933,
         0.1719291 ,  0.70525317,  0.70895558]], dtype=float64)

In [9]:
import jax.numpy as jnp
import exciting_environments as excenvs
from exciting_environments import EnvironmentRegistry
from exciting_environments.utils import MinMaxNormalization

env = EnvironmentRegistry.PENDULUM.make(
    batch_size=5,
    action_normalizations={"torque": MinMaxNormalization(min=-15,max=15)},
    tau=2e-2
) 
obs, state = env.vmap_reset()

actions = jnp.linspace(start=-1, stop=1, num=1000)[None, :, None]
actions = actions.repeat(env.batch_size, axis=0)

observations = []
observations.append(obs)

for idx in range(actions.shape[1]):
    obs, state = env.vmap_step(
        state, actions[:, idx, :]
    )
    observations.append(obs)
observations = jnp.stack(observations, axis=1)

ImportError: cannot import name 'EnvironmentRegistry' from 'exciting_environments' (/home/olivers/exc_env/exciting-environments/exciting_environments/__init__.py)